# Apprentissage Supervisé - Régression

*Lorenzo Brucato - Alexandre Loret*

In [56]:
import pandas as pd
import pyarrow as pa
import numpy as np

In [36]:
Xy_train = pd.read_parquet("data/train.parquet")
X_test = pd.read_parquet("data/test.parquet")

In [27]:
Xy_train

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee,PU_location_lat,PU_location_lon,DO_location_lat,DO_location_lon
0,1,2024-01-15 20:37:28,2024-01-15 20:56:43,1.0,3.30,1.0,N,1,19.1,3.5,0.5,4.80,0.0,1.0,2.5,0.0,40.734576,-74.002875,40.768615,-73.965635
1,2,2024-01-25 19:41:24,2024-01-25 19:44:42,1.0,0.72,1.0,N,1,5.8,2.5,0.5,2.46,0.0,1.0,2.5,0.0,40.768615,-73.965635,40.780436,-73.957012
2,2,2024-01-25 15:48:48,2024-01-25 16:24:23,2.0,7.45,1.0,N,1,38.7,0.0,0.5,3.00,0.0,1.0,2.5,0.0,40.717773,-74.007880,40.780436,-73.957012
3,2,2024-01-27 00:47:26,2024-01-27 00:55:55,1.0,1.53,1.0,N,1,10.0,1.0,0.5,2.25,0.0,1.0,2.5,0.0,40.720889,-73.996919,40.747746,-73.978492
4,2,2024-01-28 13:14:26,2024-01-28 13:22:34,1.0,1.53,1.0,N,1,10.0,0.0,0.5,3.50,0.0,1.0,2.5,0.0,40.768615,-73.965635,40.747746,-73.978492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2,2024-01-19 09:12:12,2024-01-19 09:38:07,1.0,4.38,1.0,N,1,26.1,0.0,0.5,3.00,0.0,1.0,2.5,0.0,40.768615,-73.965635,40.718938,-73.990896
99996,2,2024-01-25 00:07:40,2024-01-25 00:25:13,1.0,5.34,1.0,N,1,25.4,1.0,0.5,6.08,0.0,1.0,2.5,0.0,40.748497,-73.992438,40.706808,-74.007496
99997,1,2024-01-08 09:27:23,2024-01-08 10:01:52,1.0,10.10,99.0,N,1,35.5,0.0,0.5,0.00,0.0,1.0,0.0,0.0,40.857779,-73.885867,40.773633,-73.981532
99998,2,2024-01-06 18:42:49,2024-01-06 19:23:57,1.0,5.82,1.0,N,2,36.6,0.0,0.5,0.00,0.0,1.0,2.5,0.0,40.780436,-73.957012,40.777427,-73.905407


In [28]:
Xy_train.shape, X_test.shape

((100000, 20), (2627163, 19))

### Prétraitement

**=== NaN ? ===**

In [22]:
pd.isnull(Xy_train).sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
congestion_surcharge     0
Airport_fee              0
PU_location_lat          0
PU_location_lon          0
DO_location_lat          0
DO_location_lon          0
dtype: int64

In [23]:
pd.isnull(X_test).sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tolls_amount             0
improvement_surcharge    0
congestion_surcharge     0
Airport_fee              0
PU_location_lat          0
PU_location_lon          0
DO_location_lat          0
DO_location_lon          0
dtype: int64

**=== one-hot ===**

In [37]:
Xy_train = pd.get_dummies(Xy_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

In [38]:
Xy_train

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee,PU_location_lat,PU_location_lon,DO_location_lat,DO_location_lon,store_and_fwd_flag_Y
0,1,2024-01-15 20:37:28,2024-01-15 20:56:43,1.0,3.30,1.0,1,19.1,3.5,0.5,4.80,0.0,1.0,2.5,0.0,40.734576,-74.002875,40.768615,-73.965635,False
1,2,2024-01-25 19:41:24,2024-01-25 19:44:42,1.0,0.72,1.0,1,5.8,2.5,0.5,2.46,0.0,1.0,2.5,0.0,40.768615,-73.965635,40.780436,-73.957012,False
2,2,2024-01-25 15:48:48,2024-01-25 16:24:23,2.0,7.45,1.0,1,38.7,0.0,0.5,3.00,0.0,1.0,2.5,0.0,40.717773,-74.007880,40.780436,-73.957012,False
3,2,2024-01-27 00:47:26,2024-01-27 00:55:55,1.0,1.53,1.0,1,10.0,1.0,0.5,2.25,0.0,1.0,2.5,0.0,40.720889,-73.996919,40.747746,-73.978492,False
4,2,2024-01-28 13:14:26,2024-01-28 13:22:34,1.0,1.53,1.0,1,10.0,0.0,0.5,3.50,0.0,1.0,2.5,0.0,40.768615,-73.965635,40.747746,-73.978492,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2,2024-01-19 09:12:12,2024-01-19 09:38:07,1.0,4.38,1.0,1,26.1,0.0,0.5,3.00,0.0,1.0,2.5,0.0,40.768615,-73.965635,40.718938,-73.990896,False
99996,2,2024-01-25 00:07:40,2024-01-25 00:25:13,1.0,5.34,1.0,1,25.4,1.0,0.5,6.08,0.0,1.0,2.5,0.0,40.748497,-73.992438,40.706808,-74.007496,False
99997,1,2024-01-08 09:27:23,2024-01-08 10:01:52,1.0,10.10,99.0,1,35.5,0.0,0.5,0.00,0.0,1.0,0.0,0.0,40.857779,-73.885867,40.773633,-73.981532,False
99998,2,2024-01-06 18:42:49,2024-01-06 19:23:57,1.0,5.82,1.0,2,36.6,0.0,0.5,0.00,0.0,1.0,2.5,0.0,40.780436,-73.957012,40.777427,-73.905407,False


**=== Dates ===**

In [39]:
Xy_train['tpep_pickup_datetime'] = pd.to_numeric(pd.to_datetime(Xy_train['tpep_pickup_datetime']))
Xy_train['tpep_dropoff_datetime'] = pd.to_numeric(pd.to_datetime(Xy_train['tpep_dropoff_datetime']))
X_test['tpep_pickup_datetime'] = pd.to_numeric(pd.to_datetime(X_test['tpep_pickup_datetime']))
X_test['tpep_dropoff_datetime'] = pd.to_numeric(pd.to_datetime(X_test['tpep_dropoff_datetime']))

In [40]:
Xy_train

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee,PU_location_lat,PU_location_lon,DO_location_lat,DO_location_lon,store_and_fwd_flag_Y
0,1,1705351048000000,1705352203000000,1.0,3.30,1.0,1,19.1,3.5,0.5,4.80,0.0,1.0,2.5,0.0,40.734576,-74.002875,40.768615,-73.965635,False
1,2,1706211684000000,1706211882000000,1.0,0.72,1.0,1,5.8,2.5,0.5,2.46,0.0,1.0,2.5,0.0,40.768615,-73.965635,40.780436,-73.957012,False
2,2,1706197728000000,1706199863000000,2.0,7.45,1.0,1,38.7,0.0,0.5,3.00,0.0,1.0,2.5,0.0,40.717773,-74.007880,40.780436,-73.957012,False
3,2,1706316446000000,1706316955000000,1.0,1.53,1.0,1,10.0,1.0,0.5,2.25,0.0,1.0,2.5,0.0,40.720889,-73.996919,40.747746,-73.978492,False
4,2,1706447666000000,1706448154000000,1.0,1.53,1.0,1,10.0,0.0,0.5,3.50,0.0,1.0,2.5,0.0,40.768615,-73.965635,40.747746,-73.978492,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2,1705655532000000,1705657087000000,1.0,4.38,1.0,1,26.1,0.0,0.5,3.00,0.0,1.0,2.5,0.0,40.768615,-73.965635,40.718938,-73.990896,False
99996,2,1706141260000000,1706142313000000,1.0,5.34,1.0,1,25.4,1.0,0.5,6.08,0.0,1.0,2.5,0.0,40.748497,-73.992438,40.706808,-74.007496,False
99997,1,1704706043000000,1704708112000000,1.0,10.10,99.0,1,35.5,0.0,0.5,0.00,0.0,1.0,0.0,0.0,40.857779,-73.885867,40.773633,-73.981532,False
99998,2,1704566569000000,1704569037000000,1.0,5.82,1.0,2,36.6,0.0,0.5,0.00,0.0,1.0,2.5,0.0,40.780436,-73.957012,40.777427,-73.905407,False


**=== split X/y ===**

In [41]:
X_train = Xy_train.drop("tip_amount",axis=1)
y_train = Xy_train["tip_amount"]

X_train.shape, y_train.shape

((100000, 19), (100000,))

### Entraînement

In [42]:
from sklearn.linear_model import LinearRegression

In [43]:
reg = LinearRegression().fit(X_train, y_train)

In [61]:
# pred = pd.DataFrame(reg.predict(X_test),columns=['tip_amount'])

pred = reg.predict(X_test)

In [66]:
# pred.to_parquet('predictions/pred.parquet',index=True)

pa_table = pa.table({"row_ID": np.arange(len(pred)), "tip_amount": pred})
pa_table

pyarrow.Table
row_ID: int32
tip_amount: double
----
row_ID: [[0,1,2,3,4,...,2627158,2627159,2627160,2627161,2627162]]
tip_amount: [[8.18827774289855,3.083523057656521,3.5388618507557226,4.279976287938098,3.185143270179765,...,3.0113099053239125,2.4485330144983095,4.775359716751495,3.0419709545782325,2.816993227805142]]

In [ ]:
pa.parquet.write_table(pa_table, "predictions/test.parquet")